# Extracting browser bookmarks
> Extracting and storing bookmarks from potentially all kind of browsers (currently chrome and edge supported)

In [1]:
#|hide
#|default_exp bookmarks.browser.broswe_bookmarks_handler

In [2]:
#|hide
%load_ext autoreload
%autoreload 2

In [3]:
#|export
import os
import sys
import json
from pathlib import Path
from typing import Dict, Optional, List
from datetime import date, datetime, timedelta


from fastcore.all import patch
from pydantic import BaseModel, Field

Schema of how the bookmarks shall be stored:

In [4]:
#|export
class BookmarkModel(BaseModel):
    """
    A model representing a bookmark.
    """

    name: str = Field(..., description="The name of the bookmark")
    url: str = Field(..., description="The URL of the bookmark")
    folder: str = Field(..., description="The folder of the bookmark")
    date_added: date = Field(..., description="The date the bookmark was added")

    def __str__(self):
        return f"{self.name} ({self.url}) added on {self.date_added}"

    __repr__ = __str__

Browser base model (to be inherited by all supported browsers)

In [5]:
#|export
class BaseBrowser(BaseModel):
    """
    Base class for interacting with web browsers' bookmarks
    """
    name: str = Field(..., description="The name of the browser")
    user_data_paths: Dict[str, str] = Field(..., description="Paths to user data for different OS")
    bookmarks_file_name: str = Field(..., description="Name of the bookmarks file")
    operating_system: Optional[str] = Field(None, description="The current operating system")
    user_data_dir_path: Optional[Path] = Field(None, description="Path to the user data directory")

    def __init__(self, **data):
        super().__init__(**data)
        self._set_operating_system()
        self._set_user_data_dir_path()

    def _set_operating_system(self):
        if "win" in sys.platform.lower():
            self.operating_system = "Windows"
        elif "mac" in sys.platform.lower():
            self.operating_system = "Mac"
        elif "linux" in sys.platform.lower():
            self.operating_system = "Linux"
        else:
            self.operating_system = "Unknown"

    def _set_user_data_dir_path(self):
        if self.operating_system in self.user_data_paths:
            self.user_data_dir_path = Path(os.path.expanduser(self.user_data_paths[self.operating_system]))
        else:
            raise ValueError(f"Unsupported operating system: {self.operating_system}")

    def get_bookmarks_files(self):
        chrome_path = Path(self.user_data_dir_path)
        # find all bookmarks files
        bookmarks_files = list(chrome_path.glob("**/*Bookmarks"))
        # Ignore the snapshot files
        bookmarks_files = [
            file.as_posix() for file in bookmarks_files if "Snapshot" not in file.as_posix()
        ]
        return bookmarks_files

    def extract_bookmark_info(
        self,
        bookmarks_file_path: str,  # path to the bookmarks file
    ) -> list[BookmarkModel]:
        """Extracts bookmark information from a Chrome Bookmarks file."""
        with open(bookmarks_file_path, "r", encoding="utf-8") as f:
            bookmarks_data = json.load(f)

        bookmarks = []

        def traverse_bookmarks(node, folder=None):
            """Recursively traverses the bookmark tree."""
            if node["type"] == "url":
                date_added = self._convert_timestamp(node["date_added"])
                bookmarks.append(
                    BookmarkModel(
                        name=node["name"],
                        url=node["url"],
                        folder=folder,
                        date_added=date_added,
                    )
                )
            elif node["type"] == "folder":
                for child in node["children"]:
                    traverse_bookmarks(child, folder=node["name"])

        for root_name, root_node in bookmarks_data["roots"].items():
            traverse_bookmarks(root_node, folder=root_name)

        return bookmarks

    def extract_bookmarks(self) -> list[BookmarkModel]:
        """Extracts all Chrome Bookmarks from all profiles."""
        bookmarks_files = self.get_bookmarks_files()
        bookmarks = []
        for bookmarks_file in bookmarks_files:
            bookmarks.extend(self.extract_bookmark_info(bookmarks_file))
        return bookmarks

    def _convert_timestamp(self, timestamp: int) -> datetime:
        # Convert WebKit timestamp (microseconds since 1601-01-01) to datetime
        windows_epoch = datetime(1601, 1, 1)
        delta = timedelta(microseconds=int(timestamp))
        return (windows_epoch + delta).date()

Functions of class `BaseBrowser` must be over-ridden by child classes, if for some browser different handling of bookmarks is required :/

Chrome browser:

In [6]:
#|export
class ChromeBrowser(BaseBrowser):
    name: str = "Chrome"
    user_data_paths: Dict[str, str] = {
        "Windows": r"~\AppData\Local\Google\Chrome\User Data",
        "Mac": r"~/Library/Application Support/Google/Chrome",
        "Linux": r"~/.config/google-chrome"
    }
    bookmarks_file_name: str = "Bookmarks"

Seeing if it works fine:

In [ ]:
chrome = ChromeBrowser()

print(f"{chrome.name} Bookmarks:")
for bookmark in chrome.extract_bookmarks():
    print(bookmark)

Edge browser:

In [8]:
#|export
class EdgeBrowser(BaseBrowser):
    name: str = "Microsoft Edge"
    user_data_paths: Dict[str, str] = {
        "Windows": r"~\AppData\Local\Microsoft\Edge\User Data",
        "Mac": r"~/Library/Application Support/Microsoft Edge",
        "Linux": r"~/.config/microsoft-edge"
    }
    bookmarks_file_name: str = "Bookmarks"

In [ ]:
edge = EdgeBrowser()

print(f"{edge.name} Bookmarks:")
for bookmark in edge.extract_bookmarks():
    print(bookmark)

Brave:

In [ ]:
class BraveBrowser(BaseBrowser):
    name: str = "Brave"
    user_data_paths: Dict[str, str] = {
        "Windows": r"~\AppData\Local\BraveSoftware\Brave-Browser",
        "Mac": r"~/Library/Application Support/BraveSoftware/Brave-Browser",
        "Linux": r"~/.config/BraveSoftware/Brave-Browser"
    }
    bookmarks_file_name: str = "Bookmarks"